# Got this working in Colab
- need to adjust how modules are loaded for local use.

In [ ]:
from det.distributions import NegativeBinomial
from det.trainer import LossSlopeLRScheduler, Trainer

import patsy
import scanpy as sc
from tqdm import trange, tqdm

adata = sc.datasets.pbmc3k_processed()
adata.layers["counts"] = sc.datasets.pbmc3k()[adata.obs.index, adata.var.index].X.todense()

X = patsy.dmatrix(
    formula_like = "0 + C(louvain)",
    data = adata.obs,
    return_type = "dataframe"
)

y = pd.DataFrame(
    adata.layers["counts"],
    index=adata.obs_names,
    columns=adata.var_names
)

nb_model = NegativeBinomial()
nb_model.setup(X, y)

# — optimizer —
optimizer = torch.optim.Adam(list(nb_model.parameters.values()), lr=1.0)

# — optional ReduceLROnPlateau —
plateau = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5, threshold=1e-3
)

# — adaptive wrapper —
sched = LossSlopeLRScheduler(
    optimizer,
    plateau_scheduler=plateau,   # pass None if you only want slope-based control
    delta_low=1e-3,              # shrink LR when improvements are tiny
    delta_high=1e-1,             # grow LR when improvements are big
    decrease_factor=0.9,
    increase_factor=1.1,
    min_lr=1e-5,
    max_lr=0.5,                  # cap at 0.5 since you said it's safe
    warmup_steps=3,
    cooldown_steps=2,
)

# — loss closure —
def loss_fn():
    return -nb_model.log_likelihood(X, y)

# — train —
Trainer(optimizer, sched, loss_fn).fit(epochs=10, show_tqdm=True)